In [ ]:
# from datasets import Dataset, DatasetDict, ClassLabel
# import numpy as np

# # Define your data dictionary
# data_dict = {
#     "image": [np.random.random((128, 128, 3)), np.random.random((128, 128, 3)), np.random.random((128, 128, 3))],
#     "label": [0, 1, 2]  # Assuming label indices correspond to class labels
# }

# # Define the ClassLabel mapping
# class_label = ClassLabel(num_classes=3, names=["Cat", "Dog", "Bird"])

# Create a Dataset using DatasetDict
# dataset = DatasetDict({
#     "train": Dataset.from_dict(data_dict, features={"image": "image", "label": class_label})
# })

# # Access and print the dataset
# print(dataset)


/Users/eilaarich-landkof-stanford/miniconda3/envs/transformers/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# class_label.str2int("progressor")

In [29]:
import tifffile
# Define the ClassLabel mapping
from datasets import Dataset, DatasetDict, ClassLabel

class_label = ClassLabel(num_classes=2, names=["nonprogressor", "progressor"])

# call this for each one of the labels
def load_47_channel_tif_to_dataset_object(list_of_images_files, label):
    data_dict = {
        "image" :[],
        "labels": []
    }
    # data_dict = {
    # "image": [image_data_1, image_data_2, image_data_3],
    # "label": labels
    # }
    for image_path in list_of_images_files:
        # Read the image
        image_data = tifffile.imread(image_path)
        # Check the shape of the image data
        print("Image shape:", image_data.shape)  # Should be (47, 1024, 1024)
        data_dict["image"].append([image_data])
        data_dict["labels"].append([class_label.str2int(label)])
    return(data_dict)
    # print(data_dict)
        